In [1]:
!nvidia-smi

Wed Aug 24 13:42:37 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Aesop's fables

Using Aesop's fables as a training set to generate text. 

In [4]:
!wget http://www.gutenberg.org/files/49010/49010-0.txt
text_path = "49010-0.txt"

--2022-08-24 13:42:41--  http://www.gutenberg.org/files/49010/49010-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.gutenberg.org/files/49010/49010-0.txt [following]
--2022-08-24 13:42:42--  https://www.gutenberg.org/files/49010/49010-0.txt
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 185303 (181K) [text/plain]
Saving to: ‘49010-0.txt’

49010-0.txt         100%[===================>] 180.96K   235KB/s    in 0.8s    

2022-08-24 13:42:44 (235 KB/s) - ‘49010-0.txt’ saved [185303/185303]



In [5]:
import torch
import numpy as np
import model
from tokenizer import *
from trainer import *

In [6]:
batch_size = 32
bptt_len = 64

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

my_data = LongTextData(text_path, device=DEVICE)
batches = ChunkedTextData(my_data, batch_size, bptt_len, pad_id=0)

Reading text file from: 49010-0.txt
Done.


In [7]:
# input to the network
print(batches[0][:-1].shape)
batches[0][:-1, 0]

torch.Size([64, 32])


tensor([ 0,  2,  3,  4,  5,  6,  7,  8,  9, 10,  5, 11, 12,  6, 13, 14, 12,  5,
        15, 16,  5,  8, 17,  6, 18, 19,  9,  9, 20,  6,  9, 21,  6, 22,  5, 23,
         9, 24, 25, 23,  6, 26, 27, 16, 28,  5, 23, 29,  6, 16, 30,  6, 31, 32,
         6, 33, 32,  6, 34, 12, 35, 11, 20, 15], device='cuda:0')

In [8]:
# target tokens to be predicted
print(batches[0][1:].shape)
batches[0][1:, 0]

torch.Size([64, 32])


tensor([ 2,  3,  4,  5,  6,  7,  8,  9, 10,  5, 11, 12,  6, 13, 14, 12,  5, 15,
        16,  5,  8, 17,  6, 18, 19,  9,  9, 20,  6,  9, 21,  6, 22,  5, 23,  9,
        24, 25, 23,  6, 26, 27, 16, 28,  5, 23, 29,  6, 16, 30,  6, 31, 32,  6,
        33, 32,  6, 34, 12, 35, 11, 20, 15,  5], device='cuda:0')

In [9]:
vocab_size = len(my_data.vocab.id_to_string)
lstm_size = 2048
embedding_size = 64

In [10]:
net = model.RNN(vocab_size = vocab_size,
                                batch_size = batch_size,
                                lstm_size = lstm_size,
                                embedding_size = embedding_size)

In [11]:
train = trainer(model=net,
                dictionary = my_data)

In [12]:
train.fit(train_set=batches)

/content/drive/MyDrive/RNN/trainer.py:65: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  nn.utils.clip_grad_norm(self.model.parameters(),self.clip)


Epoch: 1 | Loss: 7.139899253845215| Perplexity: 1261.30126953125
Epoch: 2 | Loss: 5.156662464141846| Perplexity: 173.58413696289062
Epoch: 3 | Loss: 4.427094459533691| Perplexity: 83.68790435791016
Epoch: 4 | Loss: 3.9301908016204834| Perplexity: 50.916690826416016
Epoch: 5 | Loss: 3.5348916053771973| Perplexity: 34.291297912597656
Epoch: 6 | Loss: 3.18271541595459| Perplexity: 24.112138748168945
Epoch: 7 | Loss: 2.845998764038086| Perplexity: 17.218748092651367
Epoch: 8 | Loss: 2.5455780029296875| Perplexity: 12.750596046447754
Epoch: 9 | Loss: 2.2380754947662354| Perplexity: 9.37527084350586
Epoch: 10 | Loss: 1.9450398683547974| Perplexity: 6.993911266326904
Epoch: 11 | Loss: 1.6716183423995972| Perplexity: 5.320771217346191
Epoch: 12 | Loss: 1.3822375535964966| Perplexity: 3.9838054180145264
Epoch: 13 | Loss: 1.1365567445755005| Perplexity: 3.116020679473877
Epoch: 14 | Loss: 0.9554245471954346| Perplexity: 2.599774122238159
Epoch: 15 | Loss: 0.7941458821296692| Perplexity: 2.212550

([7.139899253845215,
  5.156662464141846,
  4.427094459533691,
  3.9301908016204834,
  3.5348916053771973,
  3.18271541595459,
  2.845998764038086,
  2.5455780029296875,
  2.2380754947662354,
  1.9450398683547974,
  1.6716183423995972,
  1.3822375535964966,
  1.1365567445755005,
  0.9554245471954346,
  0.7941458821296692,
  0.6578207612037659,
  0.5373135805130005,
  0.44128310680389404,
  0.37561115622520447,
  0.31908220052719116,
  0.2802034616470337,
  0.23798485100269318,
  0.21134062111377716,
  0.18323037028312683,
  0.1599201112985611,
  0.14126871526241302,
  0.13126486539840698,
  0.11760230362415314,
  0.10253380239009857,
  0.09514065086841583,
  0.08578787744045258,
  0.0802069902420044,
  0.07200014591217041,
  0.06598801910877228,
  0.05899663269519806,
  0.053277838975191116,
  0.04842580109834671,
  0.04403448477387428,
  0.03894218057394028,
  0.036971691995859146,
  0.03290059417486191,
  0.032584842294454575,
  0.03356359153985977,
  0.03751929849386215,
  0.0433085

Generating text:

In [13]:
torch.manual_seed(10091995)
test = generate(net=net, tokenizer=my_data,size=500,beginning='The ')
print(''.join(test))

The when you have once
seen the city you will never be willing to return to this quiet place.”

[Illustration]

After being urged a long time, the Country Mouse at last agreed to go
to the city that very night. So they started off together, and about
midnight came to the great house where the City Mouse lived. In the
dining room was spread a rich feast; and the City Mouse, with many
airs and graces, ran about the table, and, picking out the nicest bits,
waited upon his country friend, who, amaze


# Music lyrics

Using music lyrics from https://github.com/efm95/beatlestxt to train RNN. 

In [14]:
!wget https://raw.githubusercontent.com/h4nkyn/beatlestxt/master/input.txt

--2022-08-24 13:51:13--  https://raw.githubusercontent.com/h4nkyn/beatlestxt/master/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2414861 (2.3M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   2.30M  --.-KB/s    in 0.04s   

2022-08-24 13:51:13 (64.5 MB/s) - ‘input.txt.1’ saved [2414861/2414861]



In [15]:
text_path = 'input.txt'
batch_size = 256
bptt_len = 128

my_data = LongTextData(text_path, device=DEVICE)
batches = ChunkedTextData(my_data, batch_size, bptt_len, pad_id=0)

vocab_size = len(my_data.vocab.id_to_string)
lstm_size = 2048
embedding_size = 64

Reading text file from: input.txt
Done.


In [16]:
net = model.RNN(vocab_size = vocab_size,
                                batch_size = batch_size,
                                lstm_size = lstm_size,
                                embedding_size = embedding_size)

train = trainer(model=net,
                dictionary = my_data)

In [17]:
train.fit(train_set=batches)

Epoch: 1 | Loss: 0.7382544875144958| Perplexity: 2.092280149459839
Epoch: 2 | Loss: 0.4755738079547882| Perplexity: 1.6089372634887695
Epoch: 3 | Loss: 0.35672810673713684| Perplexity: 1.4286473989486694
Epoch: 4 | Loss: 0.2664473056793213| Perplexity: 1.305318832397461
Epoch: 5 | Loss: 0.19611157476902008| Perplexity: 1.2166626453399658
Epoch: 6 | Loss: 0.14327369630336761| Perplexity: 1.1540457010269165
Epoch: 7 | Loss: 0.10426858812570572| Perplexity: 1.1098984479904175
Epoch: 8 | Loss: 0.07408257573843002| Perplexity: 1.0768957138061523
Epoch: 9 | Loss: 0.051810186356306076| Perplexity: 1.0531758069992065
Epoch: 10 | Loss: 0.03591282293200493| Perplexity: 1.0365654230117798
Epoch: 11 | Loss: 0.024439994245767593| Perplexity: 1.0247410535812378
Perplexity below 1.03
Epoch: 12 | Loss: 0.016983237117528915| Perplexity: 1.0171282291412354
Perplexity below 1.03
Epoch: 13 | Loss: 0.012533707544207573| Perplexity: 1.0126125812530518
Perplexity below 1.03
Perplexity below 1.03 for more the

([0.7382544875144958,
  0.4755738079547882,
  0.35672810673713684,
  0.2664473056793213,
  0.19611157476902008,
  0.14327369630336761,
  0.10426858812570572,
  0.07408257573843002,
  0.051810186356306076,
  0.03591282293200493,
  0.024439994245767593,
  0.016983237117528915,
  0.012533707544207573],
 [2.092280149459839,
  1.6089372634887695,
  1.4286473989486694,
  1.305318832397461,
  1.2166626453399658,
  1.1540457010269165,
  1.1098984479904175,
  1.0768957138061523,
  1.0531758069992065,
  1.0365654230117798,
  1.0247410535812378,
  1.0171282291412354,
  1.0126125812530518])

In [18]:
torch.manual_seed(10091995)
test = generate(net=net, tokenizer=my_data,size=500,beginning='The ')
print(''.join(test))

The that we are gonna get by,
So stop acting like a lady come and cry like baby on the outside.
On the outside
On the outside

Misfits_EOT
 
You've been sleeping in a field but you look real rested
You set out to outrage but you can't get arrested
You say your image is new, but it looks well tested
You're lost without a crowd yet you go your own way

You say your summer has gone
Now the winter is crawlin' in
They say that even in your day
Somehow you never could quite fit in
Though it's cold out
